In [1]:
from scipy import interpolate
import matplotlib.pyplot as plt
import numpy as np 
import numpy as np
import torch
from torch.utils.data.dataset import TensorDataset
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
import numpy as np
import scipy.linalg as slin
import scipy.sparse as sp
import networkx as nx
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
import glob
import re
import math
from torch.optim.adam import Adam
from utils import *
from statistics import mean
import random


Bad key "text.kerning_factor" on line 4 in
D:\Anaconda\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def simulate_lsem_dynamic(W_all,Z: nx.DiGraph,
                 n: int,n_time:int, treatment_type: str,
                 noise_scale: float = 0.5,
                 baseline: float = 1.0) -> np.ndarray:
    """Simulate samples from LSEM.
        
        Args:
        W_all,A: weigthed DAG for instaneous relation and lagged relation
        n: number of samples in each time-stamp
        lag: degree of AR
        n_time: number of time stamp
        treatment_type: the type of the exposure {Binary, Gaussian}
        noise_scale: noise scale parameter of Gaussian distribution in the lSEM
        baseline: the baseline for the outcome
        
        Returns:
        X: [time,n, d] sample matrix
        """
    #W_array = nx.to_numpy_array(W)
    Z_array = nx.to_numpy_array(Z)
    d = Z_array.shape[0]
    #X_all = np.zeros([n_time+1,n, d])
    
    ## create the initial data
    X = np.zeros([n, d])
    W_0=W_all[0,:,:]
    ordered_vertices = list(nx.topological_sort(nx.from_numpy_matrix(W_0,create_using=nx.DiGraph)))
    assert len(ordered_vertices) == d
    rank_A = ordered_vertices.index(0)
    for j in ordered_vertices:
        if ordered_vertices.index(j) > rank_A:
            parents = list(nx.from_numpy_matrix(W_0,create_using=nx.DiGraph).predecessors(j))
            X[:, j] = X[:, parents].dot(W_0[parents, j]) + np.random.normal(scale=noise_scale, size=n)
        elif ordered_vertices.index(j) < rank_A:
            X[:, j] = np.random.normal(scale=noise_scale, size=n)
        else:
            if treatment_type == 'Binary':
                X[:, j] = 2 * (np.random.binomial(1, 0.5, n) - 0.5)
            elif treatment_type == 'Gaussian':
                X[:, j] = np.random.normal(scale=noise_scale, size=n)
            else:
                raise ValueError('unknown exposure type')
    X[:, d-1] += baseline
    X_all=X
    ## for follow-up time-stamps, X=XW+AZ+E
    for time in range(1,n_time+1):
        X_temp = np.matmul(X,Z_array)
        W_array=W_all[time,:,:] ## different index!
        W=nx.from_numpy_matrix(W_array,create_using=nx.DiGraph)
        for j in ordered_vertices:
            if ordered_vertices.index(j) > rank_A:
                parents = list(W.predecessors(j))
                X_temp[:, j] += X_temp[:, parents].dot(W_array[parents, j]) + np.random.normal(scale=noise_scale, size=n)
            elif ordered_vertices.index(j) < rank_A:
                X_temp[:, j] += np.random.normal(scale=noise_scale, size=n)
            else:
                if treatment_type == 'Binary':
                    X_temp[:, j] += 2 * (np.random.binomial(1, 0.5, n) - 0.5)
                elif treatment_type == 'Gaussian':
                    X_temp[:, j] += np.random.normal(scale=noise_scale, size=n)
                else:
                    raise ValueError('unknown exposure type')
        X_all=np.append(X_all,X_temp,axis=0)
        X=X_temp
    return X_all

In [3]:
W_all=np.zeros([11,5, 5])
import math 
def cos(x):
    return((-15+(5-x)**2)/15)
for i in range(11):
    W_all[i,0,4]=cos(i)

In [4]:
## create Z matrix
Z=np.identity(5)
Z[0,0]=0 # no correlation for treatment
Z_graph=nx.from_numpy_matrix(Z,create_using=nx.DiGraph)

In [6]:
def notears_linear(X, lambda1, loss_type, max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3):
    """Solve min_W L(W; X) + lambda1 ‖W‖_1 s.t. h(W) = 0 using augmented Lagrangian.

    Args:
        X (np.ndarray): [n, d] sample matrix
        lambda1 (float): l1 penalty parameter
        loss_type (str): l2, logistic, poisson
        max_iter (int): max num of dual ascent steps
        h_tol (float): exit if |h(w_est)| <= htol
        rho_max (float): exit if rho >= rho_max
        w_threshold (float): drop edge if |weight| < threshold

    Returns:
        W_est (np.ndarray): [d, d] estimated DAG
    """
    def _loss(W):
        """Evaluate value and gradient of loss."""
        M = X @ W
        if loss_type == 'l2':
            R = X - M
            loss = 0.5 / X.shape[0] * (R ** 2).sum()
            G_loss = - 1.0 / X.shape[0] * X.T @ R
        elif loss_type == 'logistic':
            loss = 1.0 / X.shape[0] * (np.logaddexp(0, M) - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (sigmoid(M) - X)
        elif loss_type == 'poisson':
            S = np.exp(M)
            loss = 1.0 / X.shape[0] * (S - X * M).sum()
            G_loss = 1.0 / X.shape[0] * X.T @ (S - X)
        else:
            raise ValueError('unknown loss type')
        return loss, G_loss

    def _h(W):
        """Evaluate value and gradient of acyclicity constraint."""
        E = slin.expm(W * W)  # (Zheng et al. 2018)
        h = np.trace(E) - d
        #     # A different formulation, slightly faster at the cost of numerical stability
        #     M = np.eye(d) + W * W / d  # (Yu et al. 2019)
        #     E = np.linalg.matrix_power(M, d - 1)
        #     h = (E.T * M).sum() - d
        G_h = E.T * W * 2
        return h, G_h

    def _adj(w):
        """Convert doubled variables ([2 d^2] array) back to original variables ([d, d] matrix)."""
        return (w[:d * d] - w[d * d:]).reshape([d, d])

    def _func(w):
        """Evaluate value and gradient of augmented Lagrangian for doubled variables ([2 d^2] array)."""
        W = _adj(w)
        loss, G_loss = _loss(W)
        h, G_h = _h(W)
        obj = loss + 0.5 * rho * h * h + alpha * h + lambda1 * w.sum()
        G_smooth = G_loss + (rho * h + alpha) * G_h
        g_obj = np.concatenate((G_smooth + lambda1, - G_smooth + lambda1), axis=None)
        return obj, g_obj

    n, d = X.shape
    w_est, rho, alpha, h = np.zeros(2 * d * d), 1.0, 0.0, np.inf  # double w_est into (w_pos, w_neg)
    bnds = [(0, 0) if i == j else (0, None) for _ in range(2) for i in range(d) for j in range(d)]
    if loss_type == 'l2':
        X = X - np.mean(X, axis=0, keepdims=True)
    for _ in range(max_iter):
        w_new, h_new = None, None
        while rho < rho_max:
            sol = sopt.minimize(_func, w_est, method='L-BFGS-B', jac=True, bounds=bnds)
            w_new = sol.x
            h_new, _ = _h(_adj(w_new))
            if h_new > 0.25 * h:
                rho *= 10
            else:
                break
        w_est, h = w_new, h_new
        alpha += rho * h
        if h <= h_tol or rho >= rho_max:
            break
    W_est = _adj(w_est)
    W_est[np.abs(W_est) < w_threshold] = 0
    return W_est

In [11]:
import scipy.optimize as sopt
n_times=30#no. of replicates
time_stamp=10 #no. of timestamp
n_var=5
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list_new=np.zeros((n_times,time_stamp+1,n_var,n_var))

for replicate in range(n_times):
  seed=seed_list[replicate]
  X_all=simulate_lsem_dynamic(W_all,Z_graph,30,10, 'Binary',noise_scale=0.1).reshape(330,5,1) #create data
  average_list=np.zeros((time_stamp+1,n_var, n_var))
  for time in range(time_stamp+1):
    X=X_all.reshape(330,5)[30*time:(30*time+30)]
    average_list[time,:,:]=notears_linear(X, lambda1=0.1, loss_type="l2", max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3)
    average_coef_list_new[replicate,:,:,:]=average_list

In [15]:
np.save("quadra_10_30_NOTEARS_lag",average_coef_list_new)

In [16]:
W_all=np.zeros([11,5, 5])
def cos(x):
    return(math.cos(x/4*math.pi)*0.8)
for i in range(11):
    W_all[i,0,4]=cos(i)

In [17]:
import scipy.optimize as sopt
n_times=30#no. of replicates
time_stamp=10 #no. of timestamp
n_var=5
np.random.seed(1234567) #Random seed
seed_list=np.random.randint(1, 1000000, size=n_times)
average_coef_list_new=np.zeros((n_times,time_stamp+1,n_var,n_var))

for replicate in range(n_times):
  seed=seed_list[replicate]
  X_all=simulate_lsem_dynamic(W_all,Z_graph,30,10, 'Binary',noise_scale=0.1).reshape(330,5,1) #create data
  average_list=np.zeros((time_stamp+1,n_var, n_var))
  for time in range(time_stamp+1):
    X=X_all.reshape(330,5)[30*time:(30*time+30)]
    average_list[time,:,:]=notears_linear(X, lambda1=0.1, loss_type="l2", max_iter=100, h_tol=1e-8, rho_max=1e+16, w_threshold=0.3)
    average_coef_list_new[replicate,:,:,:]=average_list

In [18]:
np.save("cos_10_30_NOTEARS_lag",average_coef_list_new)